In [1]:
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.quantum_info import random_statevector, Statevector
import numpy as np
sim = Aer.get_backend('aer_simulator')  # Tell Qiskit how to simulate our circuit

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
def evolve_1q(ini_vec,op): # evolves 1 qubit initial sate by applying one pauli gate
    qc = QuantumCircuit(1)  # Create a quantum circuit with one qubit
    qc.initialize(ini_vec, 0) # Apply initialisation operation to the 0th qubit
    if op=='x': qc.x(0)
    if op=='y': qc.y(0)
    if op=='z': qc.z(0)
    print(qc)
    return qc

In [3]:
def hadamOverlap_2q(ivecL,opL,type='re'): # Hadamard-overlap for 2 qubita initial state and one pauli gate
    qc = QuantumCircuit(3) 
    qc.h(0)
    if type=='im': qc.sdg(0)  # will compute imaginary component
    for i in range(2):
        qc.initialize(ivecL[i], i+1) # Apply initialisation 
        op=opL[i]
        if op=='x': qc.cx(0,i+1)
        elif op=='y': qc.cy(0,i+1)
        elif op=='z': qc.cz(0,i+1)    
        else:  bad_op
    qc.h(0)
    print(qc)
    return qc

In [4]:
ini_vec1 = random_statevector(2)    
ini_vec2 = random_statevector(2)  
Uop='zy'  # choose unitary operators for both qubits: x,y,z
print('ini_vec1:',ini_vec1.data,'\nini_vec2:',ini_vec2.data,'Uop:',Uop )

ini_vec1: [-0.11026209-0.40442706j -0.85724807+0.29900964j] 
ini_vec2: [0.36448251+0.38405974j 0.13992355-0.83670306j] Uop: zy


In [5]:
# use Qiskit’s simulators apply two different us on two different initial states
qc=evolve_1q(ini_vec1,op=Uop[0])
qc.save_statevector('o1')
result = sim.run(qc).result() # Do the simulation and return the result
ov1 = result.data(0)['o1']
print('out vec 1', ov1.data)
over1=ini_vec1.inner(ov1)
print('TRUTH_A: <iv1|ov1>= ',over1)

   ┌─────────────────────────────────────────────────┐┌───┐
q: ┤ Initialize(-0.11026-0.40443j,-0.85725+0.29901j) ├┤ Z ├
   └─────────────────────────────────────────────────┘└───┘
out vec 1 [-0.11026209-0.40442706j  0.85724807-0.29900964j]
TRUTH_A: <iv1|ov1>=  (-0.6485620443114151+0j)


In [6]:
qc=evolve_1q(ini_vec2,op=Uop[1])
qc.save_statevector('o2')
result = sim.run(qc).result() # Do the simulation and return the result
ov2 = result.data(0)['o2']
print('out vec 2', ov2.data)
over2=ini_vec2.inner(ov2)
print('TRUTH_B: <iv2|ov2>= ',over2)

   ┌──────────────────────────────────────────────┐┌───┐
q: ┤ Initialize(0.36448+0.38406j,0.13992-0.8367j) ├┤ Y ├
   └──────────────────────────────────────────────┘└───┘
out vec 2 [-0.83670306-0.13992355j -0.38405974+0.36448251j]
TRUTH_B: <iv2|ov2>=  (-0.7174052698816521+0j)


In [7]:
over12=over1*over2
print('TRUTH: <iv1|ov1>*<iv2|ov2>= ',over12)

TRUTH: <iv1|ov1>*<iv2|ov2>=  (0.4652818284342267-0j)


In [8]:
qc3=hadamOverlap_2q([ini_vec1,ini_vec2],opL=Uop)
qc3.save_statevector('ov3')
result3 = sim.run(qc3).result() # Do the simulation and return the result
ov3 = result3.data(0)['ov3']
print('out vec3 len:', len(ov3.data))
p0,p1=ov3.probabilities([0])
print('p0,p1:',p0,p1, 'p0-p1:',p0-p1)

                            ┌───┐                               ┌───┐
q_0: ───────────────────────┤ H ├────────────────────────■───■──┤ H ├
     ┌──────────────────────┴───┴──────────────────────┐ │   │  └───┘
q_1: ┤ Initialize(-0.11026-0.40443j,-0.85725+0.29901j) ├─■───┼───────
     └─┬──────────────────────────────────────────────┬┘   ┌─┴─┐     
q_2: ──┤ Initialize(0.36448+0.38406j,0.13992-0.8367j) ├────┤ Y ├─────
       └──────────────────────────────────────────────┘    └───┘     
out vec3 len: 8
p0,p1: 0.7326409142171132 0.2673590857828867 p0-p1: 0.4652818284342265


In [9]:
diff=np.real(over12)-(p0-p1)
if abs(diff)<1e-4:
    print('CHECK Real PASSED diff=%.3f  Uop=%s  over12=%s'%(diff,Uop,over12))
else:
    print('CHECK Real *** FAILED***',diff)  

CHECK Real PASSED diff=0.000  Uop=zy  over12=(0.4652818284342267-0j)
